In [1]:
import json
import pandas as pd
import torch
from eval_F1 import f1_score,metric_max_over_ground_truths
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import numpy as np 
import os 
from rouge import Rouge
from tqdm import tqdm 
rouge = Rouge()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

def prepare_prompt_query(context,entity):
    # with cosmo
    prompt = f"You are given a short dialog between a user and a bot for a discussion on {entity}. Convert the bot response to a question to use for internet search to get relevant knowledge for continuing the response.\n\n"
    # # # without cosmo
    # prompt = f"You are given a short dialog between a user and a bot for a discussion on {entity}. For the next bot response, generate a search query to use for the internet\n\n"
    prompt += context
    prompt += "\n\nquestion:"
    
    # prompt = context + "\n\nquestion:"
    
    return prompt

def prepare_prompt_entity(context:str):
    final_prompt = "You are given a dialog between bot and user.  You need to identify the current Topic being discussed.\n\n"+ context + "\n\nTopic: "
    return final_prompt

def prepare_prompt_entity_train(context):
    turns = len(context)
    dialog_text =""
    for i in range(turns):
        dialog_text += "\nUser1: " + context[i]["user1"] + "\nUser2: "+context[i]["user2"]
    final_prompt = "You are given a dialog between user1 and user2.  You need to identify the current Topic being discussed.\n\n"+ dialog_text + "\n\nTopic: "
    return final_prompt

def query_inference(context: str, model,entity:str):
    input_text = prepare_prompt_query(context,entity)
    # print(input_text)
    input_ids = tokenizer(input_text,return_tensors = 'pt').input_ids.cuda()
    outputs = model.generate(input_ids, max_new_tokens=40, num_beams=4)
    query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(query)
    return query

def entity_inference(context,model):
    input_text = prepare_prompt_entity_train(context)
    input_ids = tokenizer(input_text,return_tensors = 'pt').input_ids
    # input_ids = tokenizer(input_text,return_tensors = 'pt').input_ids.cuda()

    outputs = model.generate(input_ids, max_new_tokens=15, num_beams=4)

    entity = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return entity 
    

def evaluate_query(idx:int,dataset,model):
    context = dataset[idx]['context']
    entity = dataset[idx]['entity']
    gt_query = dataset[idx]['query_gen']
    # inference
    query = query_inference(context,model,entity)
    # print(query)
    # print(gt_query)
    return rouge.get_scores(query, gt_query)[0]['rouge-l']['f']

def evaluate_entity(idx:int, dataset,model):
    entity = entity_inference(dataset[idx]['context'],model)
    gt_entity = dataset[idx]['predict']
    score = metric_max_over_ground_truths(f1_score, entity, [gt_entity])
    return score

def evaluate_entity_main(dataset_path,model_path):
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path).cuda()
    fp = open(dataset_path)
    dataset = json.load(fp)
    length = 1000
    score = 0
    for i in tqdm(range(length)):
        _score = evaluate_entity(i,dataset,model)
        # print(_score)
        score += _score
    score/= length
    print(score)
    return score

def evaluate_query_main(dataset_path,model_path):
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path).cuda()
    fp = open(dataset_path)
    dataset = json.load(fp)
    length = 1
    score = 0
    for i in tqdm(range(length)):
        _score = evaluate_query(i,dataset,model)
        # print(_score)
        score += _score
    score/= length
    print(score)
    return score

In [2]:
def _evaluate_query(idx:int,dataset1,dataset2,model):
    context = dataset1[idx]['context']
    entity = dataset1[idx]['entity']
    gt_query = dataset2[idx]['query_gen']
    # inference
    query = query_inference(context,model,entity)
    # print(query)
    # print(gt_query)
    return rouge.get_scores(query, gt_query)[0]['rouge-l']['f']

model = AutoModelForSeq2SeqLM.from_pretrained("../query_cosmo_20k/")
fp1 = open("../dataset/train_query.json")
fp2 = open("../dataset/train_query_nocosmo.json")
dataset1 = json.load(fp1)
dataset2 = json.load(fp2)
length = 1000
score = 0
for i in tqdm(range(length)):
    _score = _evaluate_query(i,dataset1,dataset2,model)
    # print(_score)
    score += _score
score/= length
print(score)

100%|██████████| 1000/1000 [10:12<00:00,  1.63it/s]

0.2941630815862882


╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│  /tmp/ipykernel_3044/3179338953.py:24                                                            │
│                                                                                                  │
│ ▲                                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: 'return' outside function

In [3]:
score

0.2941630815862882